## Tool - spaCy EntityLinker
---

### Load and standardize the data


In [1]:
import pandas as pd
import ast

# input file
nel_result = '../../data/results/spacy_entity_linker/FAA_DataModel_20240104104402.csv'

# load output from the tool to be evaluated
data = pd.read_csv(nel_result)

# rename columns
data.rename(columns={"c5": "id", "c119": "sample", "c119_entity_linking":"entity_linking"}, inplace=True)

# Convert the string representation of a list of dictionaries to actual list of dictionaries
data['entity_linking'] = data['entity_linking'].apply(ast.literal_eval)

# explode the entity_linking column
data = data.explode('entity_linking').reset_index(drop=True)

# Convert the dictionary column to a DataFrame
attributes_df = data['entity_linking'].apply(pd.Series)

# change the column type to object
attributes_df['indentifier'] = attributes_df['indentifier'].astype('Int64')

attributes_df.drop(columns=[0], inplace=True)
#attributes_df

# Concatenate with the original DataFrame (minus the dictionary column)
data = pd.concat([data.drop('entity_linking', axis=1), attributes_df], axis=1)

# rename columns
data.rename(columns={"label": 'entity', "indentifier": "qid"}, inplace=True)

# save the processed tool output to a file
data.to_csv('../../tool_results/spacy_entitylinker.csv', index=False)
data

,id,sample,qid,entity,description
0,19750315005389A,TAILWHEEL COCKED RIGHT PRIOR TO TKOF. ...,8015236,William Matthew Prior,American painter
1,19750419011349A,TOW PLANE BECAME AIRBORNE THEN SETTLED.STUDENT...,6588629,Toledo Airport (Brazil),None
2,19750419011349A,TOW PLANE BECAME AIRBORNE THEN SETTLED.STUDENT...,4698432,Airborne,controversial dietary supplement
3,19750419011349A,TOW PLANE BECAME AIRBORNE THEN SETTLED.STUDENT...,3951828,agency,capacity of an agent to act in a world
4,19750419011349A,TOW PLANE BECAME AIRBORNE THEN SETTLED.STUDENT...,6588629,Toledo Airport (Brazil),None
...,...,...,...,...,...
16891,20080404840559A,(-23) THE AIRCRAFT EXPERIENCED SEVERE TURBULAN...,5421066,Experienced,live album
16892,20080404840559A,(-23) THE AIRCRAFT EXPERIENCED SEVERE TURBULAN...,6203,Avogadro constant,"fundamental physical constant (symbols: L,Nᴀ) ..."
16893,20080404840559A,(-23) THE AIRCRAFT EXPERIENCED SEVERE TURBULAN...,5727902,circa,approximately – should be used with qualifier ...
16894,20080404840559A,(-23) THE AIRCRAFT EXPERIENCED SEVERE TURBULAN...,1431062,Castellón-Costa Azahar Airport,airport


## Gold standard NEL data

### Load the processed gold standard data

In [2]:
import pandas as pd

gold_standard_path = '../../gold_standard/processed/nel.csv'

# load output from the tool to be evaluated
gs = pd.read_csv(gold_standard_path)

gs

,id,sample,entity,qid
0,19990213001379A,ACFT WAS TAXIING FOR TAKE OFF WHEN IT LOST CON...,ACFT,11436
1,19990213001379A,ACFT WAS TAXIING FOR TAKE OFF WHEN IT LOST CON...,DITCH,2048319
2,19990213001379A,ACFT WAS TAXIING FOR TAKE OFF WHEN IT LOST CON...,TREE,10884
3,19990213001379A,ACFT WAS TAXIING FOR TAKE OFF WHEN IT LOST CON...,LOST CONTROL,29017603
4,19800217031649I,"AFTER TAKEOFF, ENGINE QUIT. WING FUEL TANK SUM...",TAKEOFF,854248
...,...,...,...,...
496,19870523018729A,CANOPY CAME OPEN ON CLIMBOUT CAUSING AN UNCONT...,PILOT ERROR,3057459
497,20030620012809I,(-23) PILOT FAILED TO ASSURE THE OIL FILLER CA...,PILOT,2095549
498,20030620012809I,(-23) PILOT FAILED TO ASSURE THE OIL FILLER CA...,OIL FILLER CAP,-1
499,20030620012809I,(-23) PILOT FAILED TO ASSURE THE OIL FILLER CA...,OIL,42962


In [3]:
id = '19990213001379A'
print(gs['sample'][gs.id==id].unique())
print(data['sample'][data.id==id].unique())



['ACFT WAS TAXIING FOR TAKE OFF WHEN IT LOST CONTROL, RAN INTO A DITCH, AND STRUCK A TREE. OTHER CIRCUMSTANCES AE UNK']
['ACFT WAS TAXIING FOR TAKE OFF WHEN IT LOST CONTROL, RAN INTO A DITCH, AND STRUCK A TREE. OTHER CIRCUMSTANCES AE UNK']


In [4]:
cols = ['id','entity','qid']
gs[cols][gs.id==id]

,id,entity,qid
0,19990213001379A,ACFT,11436
1,19990213001379A,DITCH,2048319
2,19990213001379A,TREE,10884
3,19990213001379A,LOST CONTROL,29017603


In [5]:
cols = ['id','entity','qid', 'description']
data[cols][data.id==id]

,id,entity,qid,description
14221,19990213001379A,United States Army Combat Fitness Test,67935434,Physical fitness test for the United States Army
14222,19990213001379A,The Take,3989575,television series
14223,19990213001379A,OpenType Font,260180,file format
14224,19990213001379A,Control,54935655,2019 action-adventure video game developed by ...
14225,19990213001379A,Royal Australian Navy,741691,naval warfare branch of the Australian Defence...
14226,19990213001379A,Charlotte Brontë,127332,English novelist and poet
14227,19990213001379A,Adobe After Effects,83380,digital motion graphics and compositing software


### Find partial matches 


Find partial matches between the `entity` values in your `gs` and `data` DataFrames, where the `entity` from `gs` is contained within the `entity` in `data` for the same `id`.


In [11]:
# Filter out rows where 'qid' is '-1'
gs= gs[gs['qid'] != -1]
gs.shape

(396, 4)

In [12]:
# Initialize an empty list to store matching records
matches = []

# Iterate through each row in the gs dataframe
for _, gs_row in gs.iterrows():
    # Filter the data dataframe for rows with the same id
    data_filtered = data[data['id'] == gs_row['id']]
    
    # Check if the gs entity is a substring of any entity in the filtered data rows
    for _, data_row in data_filtered.iterrows():
        if gs_row['entity'].lower() in data_row['entity'].lower():
            # Add the match to the matches list with qid values from both gs and data
            match = {
                'gs_id': gs_row['id'],
                'gs_entity': gs_row['entity'],
                'gs_qid': gs_row['qid'],  # qid from gs
                'data_entity': data_row['entity'],
                'data_qid': int(data_row['qid']),  # qid from data
                'data_description': data_row['description']
            }
            matches.append(match)

# Convert the matches list to a DataFrame
matches_df = pd.DataFrame(matches)

matches_df

,gs_id,gs_entity,gs_qid,data_entity,data_qid,data_description
0,19800217031649I,TAKEOFF,854248,Takeoff,48719890,"American rapper, member of the hip hop group M..."
1,19800217031649I,ENGINE,44167,Engine,18353587,None
2,19800217031649I,WING,161358,WING,7950776,"commercial AM radio station in Dayton, Ohio"
3,19790720021329A,HELICOPTER,34486,HElicopTEr,5629507,album by Download
4,19790720021329A,TREES,10884,Trees,7837713,poem by Joyce Kilmer
...,...,...,...,...,...,...
204,19960418007829A,DEPARTURE,21171241,Departure,1922930,album by Journey
205,19870523018729A,PILOT ERROR,3057459,pilot error,3057459,"decision, action or inaction by a pilot of an ..."
206,20030620012809I,PILOT,2095549,PILOT,2044212,historic programming language
207,20030620012809I,OIL,42962,Oil,7081283,episode of The Young Ones


In [14]:
# Count True Positives where gs_qid matches data_qid
TP = sum(matches_df['gs_qid'] == matches_df['data_qid'])

# Output the count of True Positives
print(f"True Positives: {TP}")

True Positives: 28
